# CreatingDatabases and Tables

## Creating Databases
- Varies by the database type
- Databases like PostgreSQL and MySQL have
command line tools to initialize the database
- With SQLite, the create_engine() statement will
create the database and file is they do not already exist

## Building a Table

In [13]:
from sqlalchemy import (Table, Column, String, Integer, Boolean, Float)
from sqlalchemy import MetaData, create_engine

metadata = MetaData()
engine = create_engine('sqlite:///:memory:')

In [14]:
employees = Table('employees', metadata,
                  Column('id', Integer()),
                  Column('name', String(255)),
                  Column('salary', Float()),
                  Column('active', Boolean()))

metadata.create_all(engine)
engine.table_names()

['employees']

## Creating Tables
- Still uses the Table object like we did for reflection
- Replaces the autoload keyword arguments with
Column objects
- Creates the tables in the actual database by using
the create_all() method on the MetaData
instance
- You need to use other tools to handle database
table updates, such as Alembic or raw SQL

## Creating Tables - Additional Column Options
- unique forces all values for the data in a column to be
unique
- nullable determines if a column can be empty in a
row
- default sets a default value if one
isn’t supplied.

## Building a Table with Additional Options

- build the same table with some extras
- name col is unique and not allowed to be empty
- default salary of $100
- set active to be true by default


In [35]:
from sqlalchemy import (Table, Column, String, Integer, Boolean, Float)
from sqlalchemy import MetaData, create_engine
engine = create_engine('sqlite:///:memory:')
metadata = MetaData() 
connection = engine.connect()

In [36]:
   
employees = Table('employees', metadata,
                  Column('id', Integer()),
                  Column('name', String(255), unique=True,
                         nullable=False),
                  Column('salary', Float(), default=100.00),
                  Column('active', Boolean(), default=True))

In [37]:
employees.constraints

{CheckConstraint(<sqlalchemy.sql.elements.BinaryExpression object at 0x103fe4400>, name='_unnamed_', table=Table('employees', MetaData(bind=None), Column('id', Integer(), table=<employees>), Column('name', String(length=255), table=<employees>, nullable=False), Column('salary', Float(), table=<employees>, default=ColumnDefault(100.0)), Column('active', Boolean(), table=<employees>, default=ColumnDefault(True)), schema=None), _create_rule=<sqlalchemy.util.langhelpers.portable_instancemethod object at 0x104696480>, _type_bound=True),
 PrimaryKeyConstraint(),
 UniqueConstraint(Column('name', String(length=255), table=<employees>, nullable=False))}

---
# Let’s practice!

In [18]:
engine = create_engine('sqlite:///:memory:')

In [20]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, Integer, Float, Boolean, String
engine = create_engine('sqlite:///:memory:')
metadata = MetaData() 

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255)),
             Column('count', Integer()),
             Column('amount', Float()),
             Column('valid', Boolean())
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print table details
print(repr(data))

Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>), schema=None)


In [29]:
# Import Table, Column, String, Integer, Float, Boolean from sqlalchemy
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

engine = create_engine('sqlite:///:memory:')
metadata = MetaData() 
connection = engine.connect()

# Define a new table with a name, count, amount, and valid column: data
data = Table('data', metadata,
             Column('name', String(255), unique=True),
             Column('count', Integer(), default=1),
             Column('amount', Float()),
             Column('valid', Boolean(), default=False)
)

# Use the metadata to create the table
metadata.create_all(engine)

# Print the table details
print(repr(metadata.tables['data']))


Table('data', MetaData(bind=None), Column('name', String(length=255), table=<data>), Column('count', Integer(), table=<data>, default=ColumnDefault(1)), Column('amount', Float(), table=<data>), Column('valid', Boolean(), table=<data>, default=ColumnDefault(False)), schema=None)


# Inserting Datainto aTable

## Adding Data to a Table
- Done with the insert() statement
- Insert() takes the table we are loading data into as the
argument
- We add all the values we want to insert in with
the values clause as column=value pairs
- Doesn’t return any rows, so no need for a fetch method

## Inserting One Row

```python
In [1]: from sqlalchemy import insert
In [2]: stmt = insert(employees).values(id=1,
 name='Jason', salary=1.00, active=True)
In [3]: result_proxy = connection.execute(stmt)
In [4]: print(result_proxy.rowcount)
Out[4]: 1
    ```

In [38]:
employees

Table('employees', MetaData(bind=None), Column('id', Integer(), table=<employees>), Column('name', String(length=255), table=<employees>, nullable=False), Column('salary', Float(), table=<employees>, default=ColumnDefault(100.0)), Column('active', Boolean(), table=<employees>, default=ColumnDefault(True)), schema=None)

---

```pyton

from sqlalchemy import insert
stmt = insert(employees).values(id=1, 
                                name='Jason',
                                salary=1.00, 
                                active=True)

result_proxy = connection.execute(stmt)

```

## Inserting Multiple Rows
- Build an insert statement without any values
- Build a list of dictionaries that represent all the
values clauses for the rows you want to insert
- Pass both the stmt and the values list to the
execute method on connection

## Inserting Multiple Rows

```python
In [1]: stmt = insert(employees)
In [2]: values_list = [
 {'id': 2, 'name': 'Rebecca', 'salary': 2.00,
 'active': True},
 {'id': 3, 'name': 'Bob', 'salary': 0.00,
 'active': False}
 ]
In [3]: result_proxy = connection.execute(stmt,
 values_list)
In [4]: print(result_proxy.rowcount)
Out[4]: 2
    ```

---
# Let’s practice!

```python
# Import insert and select from sqlalchemy
from sqlalchemy import insert, select

# Build an insert statement to insert a record into the data table: stmt
stmt = insert(data).values(name='Anna', count=1, amount=1000.00, valid=True)

# Execute the statement via the connection: results
results = connection.execute(stmt)

# Print result rowcount
print(results.rowcount)

# Build a select statement to validate the insert
stmt = select([data]).where(data.columns.name == 'Anna')

# Print the result of executing the query.
print(connection.execute(stmt).first())
```

```python
# Build a list of dictionaries: values_list
values_list = [
    {'name': 'Anna', 'count': 1, 'amount': 1000.00, 'valid': True},
    {'name': 'Taylor', 'count': 1, 'amount': 750.00, 'valid': False}
]

# Build an insert statement for the data table: stmt
stmt = insert(data)

# Execute stmt with the values_list: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)
```


```python
# Create a insert statement for census: stmt
stmt = insert(census)

# Create an empty list and zeroed row count: values_list, total_rowcount
values_list = []
total_rowcount = 0

# Enumerate the rows of csv_reader
for idx, row in enumerate(csv_reader):
    #create data and append to values_list
    data = {'state': row[0], 'sex': row[1], 'age': row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    values_list.append(data)

    # Check to see if divisible by 51
    if idx % 51 == 0:
        results = connection.execute(stmt, values_list)
        total_rowcount += results.rowcount
        values_list = []

# Print total rowcount
print(total_rowcount)
```

---

# UpdatingDatain a Table

Updating Data in a Table
- Done with the update statement
- Similar to the insert statement but includes a where
 lause to determine what record will be updated
- We add all the values we want to update with
the values clause as column=value pairs

## Updating One Row
- employee with the `id=3` an active employee

```python
from sqlalchemy import update
stmt = update(employees)
stmt = stmt.where(employees.columns.id == 3)
stmt = stmt.values(active=True)

result_proxy = connection.execute(stmt)

print(result_proxy.rowcount)
Out[6]: 1

```

## Updating Multiple Rows
- Build a where clause that will select all the records you want
to update

## Inserting Multiple Rows

 - all active employees to inactive with $0
 
```python
stmt = update(employees)  
stmt = stmt.where(
            employees.columns.active == True
            )  # matches all active empl

# update teh active and salary with the  changes
stmt = stmt.values(active=False, salary=0.00)

# execute the satement
result_proxy = connection.execute(stmt)

print(result_proxy.rowcount)
Out[5]: 3 
```

## Correlated Updates

- pay all employees the same amount

```python

# select statement 
new_salary = select([employees.columns.salary])

# order by max salary
new_salary = new_salary.order_by(desc(
    employees.columns.salary)
 )
    
# select the maximum salary  
new_salary = new_salary.limit(1)


# create an update statement for the employees table
# Without a where clause to update every record
stmt = update(employees)


# in the value clause, we select the salary column to
# the select statement
stmt = stmt.values(salary=new_salary)

# when executed, it will find the maximum statement
# that will be used to salary field for every record
result_proxy = connection.execute(stmt)


print(result_proxy.rowcount)
Out[7]: 3
```


## Correlated Updates
- Uses a `select()` statement to find the value for the
column we are updating
- Commonly used to update records to a maximum value or
change a string to match an abbreviation from another table

---
# Let’s practice!

```python
# Build a select statement: select_stmt
select_stmt = select([state_fact]).where(state_fact.columns.name == 'New York')

# Print the results of executing the select_stmt
print(connection.execute(select_stmt).fetchall())

# Build a statement to update the fips_state to 36: stmt
stmt = update(state_fact).values(fips_state = 36)

# Append a where clause to limit it to records for New York state
stmt = stmt.where(state_fact.columns.name == 'New York')

# Execute the statement: results
results = connection.execute(stmt)

# Print rowcount
print(results.rowcount)

# Execute the select_stmt again to view the changes
print(connection.execute(select_stmt).fetchall())
```

```python
# Build a statement to update the notes to 'The Wild West': stmt
stmt = update(state_fact).values(notes='The Wild West')

# Append a where clause to match the West census region records
stmt = stmt.where(state_fact.columns.census_region_name == 'West')

# Execute the statement: results
results = connection.execute(stmt)

# Print rowcount
print(results.rowcount)```

```python
# Build a statement to select name from state_fact: stmt
fips_stmt = select([state_fact.columns.name])

# Append a where clause to Match the fips_state to flat_census fips_code
fips_stmt = fips_stmt.where(
    state_fact.columns.fips_state == flat_census.columns.fips_code)

# Build an update statement to set the name to fips_stmt: update_stmt
update_stmt = update(flat_census).values(state_name=fips_stmt)

# Execute update_stmt: results
results = connection.execute(update_stmt)

# Print rowcount
print(results.rowcount)
```

# Deleting Data from a Database

## Deleting Data from a Table
- Done with the delete() statement
- delete() takes the table we are loading data into as
the argument
- A where() clause is used to choose which rows to
delete
- Hard to undo so BE CAREFUL!!!

## Deleting all Data from a Table
- Delete ALL the data from a table called `extra_employees`

```python
from sqlalchemy import delete

# select statement to count the records in the extra_employees table
# in order to make sure we correcly delete the number of records
stmt = select([
    func.count(extra_employees.columns.id)])

# we execute thestatements and use scalar() method to get 
# the nubmer of records back
connection.execute(stmt).scalar()
Out[3]: 3
 
# create a delete statement that targets the extra_employees table
# without a where clause

delete_stmt = delete(extra_employees)

# execute the delete statement
result_proxy = connection.execute(delete_stmt)

# check the affected rows
result_proxy.rowcount
Out[6]: 3 
```

## Deleting Specific Rows
- Build a where clause that will select all the records you want
to delete


## Deleting Specific Rows

- remove employee with an `id=3` from the employees table

```python
# build a delete statement for the employees table
# using a where clause to target the employee with the id of 3
stmt = delete(employees).where(employees.columns.id == 3)

# execute the statement
result_proxy = connection.execute(stmt)

# check the rowcount to see we removed only one record
result_proxy.rowcount
Out[3]: 1
    ```

## Dropping a Table Completely
- Uses the drop method on the table
- Accepts the engine as an argument so it knows
where to remove the table from
- Won’t remove it from metadata until the python process is
restarted

## Dropping a table

- drop the extra_employees table

```python

# call the extra_employees table and pass the engine that points to our database
extra_employees.drop(engine)

# verify that it was deleted with the .exist() method to see if it still there
print(extra_employees.exists(engine))
Out[2]: False
    ```

## Dropping all the Tables
- Uses the drop_all() method on MetaData

## Dropping all the Tables

- drop all the tables

```python

# use the drop_all method on the metadata
metadata.drop_all(engine)

engine.table_names()

Out[2]: []
    ```

---
# Let’s practice!

```python
# Import delete, select
from sqlalchemy import delete, select

# Build a statement to empty the census table: stmt
stmt = delete(census)

# Execute the statement: results
results = connection.execute(stmt)

# Print affected rowcount
print(results.rowcount)

# Build a statement to select all records from the census table
stmt = select([census])

# Print the results of executing the statement to verify there are no rows
print(connection.execute(stmt).fetchall())
```


```python
# Build a statement to count records using the sex column for Men ('M') age 36: stmt
stmt = select([func.count(census.columns.sex)]).where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the select statement and use the scalar() fetch method to save the record count
to_delete = connection.execute(stmt).scalar()

# Build a statement to delete records from the census table: stmt_del
stmt_del = delete(census)

# Append a where clause to target Men ('M') age 36
stmt_del = stmt_del.where(
    and_(census.columns.sex == 'M',
         census.columns.age == 36)
)

# Execute the statement: results
results = connection.execute(stmt_del)

# Print affected rowcount and to_delete record count, make sure they match
print(results.rowcount, to_delete)
```

```python
# Drop the state_fact table
state_fact.drop(engine)

# Check to see if state_fact exists
print(state_fact.exists(engine))

# Drop all tables
metadata.drop_all(engine)

# Check to see if census exists
print(census.exists(engine))

```